# trajectory_analyze

particle trajectory analysis

file requirements:
- particle netcdf file generated by parcels after simulation
- the actual ocean data the particles ran on

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import scipy.spatial
import xarray as xr

import utils
from parcels_utils import HFRGrid
from parcels_analysis import *
from plot_utils import plot_particles, get_carree_axis, get_carree_gl, generate_simulation_plots

In [ ]:
# the data generated by ParticleFile
p_nc = utils.PARTICLE_NETCDF_DIR / "particle_plume_track.nc"
# particle data
p_ds = xr.open_dataset(p_nc)
p_ds.close()
p_results = ParticleResult(p_ds)

# ocean current netcdf data the particles used
d_nc = utils.FILES_ROOT / utils.CURRENT_NETCDF_DIR / "west_coast_1km_hourly/tj_plume_interped.nc"
# ocean current data
d_info = HFRGrid(d_nc)
d_ds = d_info.xrds
fs = d_info.fieldset

In [ ]:
lats, lons = utils.load_pts_mat(utils.MATLAB_DIR / "coastline.mat", "latz0", "lonz0")
# lats, lons = utils.load_pts_mat(utils.MATLAB_DIR / "coastline_1km.mat", "rlatz0", "rlonz0")
coastline_points = np.array([lats, lons]).T
coast_kdtree = scipy.spatial.KDTree(coastline_points)

tijuana_mouth = np.array([32.551967, -117.127208])

In [ ]:
collide_dist_thresh = 100  # meters
plume_pot_thresh = 500  # meters
feature_coast = ParticlePlotFeature.get_sd_coastline(track_dist=collide_dist_thresh)
feature_station = StationFeature.get_sd_stations(track_dist=plume_pot_thresh)
feature_mouth = LatTrackedPointFeature.get_tijuana_mouth()

In [ ]:
TIJUANA_MOUTH_DOMAIN = dict(
    S=32.53,
    N=32.564,
    W=-117.162,
    E=-117.105
)

# domain = TIJUANA_MOUTH_DOMAIN
domain = d_info.get_domain()
domain["W"] = -117.26
domain["S"] = 32.25

# line segment representation of coastline
fig, ax = get_carree_axis(domain)
get_carree_gl(ax)
plt.scatter(feature_coast.lons, feature_coast.lats, s=10)
plt.scatter(feature_station.lons, feature_station.lats, s=30)
# plt.plot(coastline_points.T[1], coastline_points.T[0])

### distance to the coastline

get particles' closest distance to shore at every position and time saved in the particle data

In [ ]:
coast_dists = np.full((p_ds.dims["traj"], p_ds.dims["obs"]), np.inf)
mouth_dists = np.full((p_ds.dims["traj"], p_ds.dims["obs"]), np.inf)
station_counts = np.zeros((p_ds.dims["obs"], len(feature_station.lats)))
for i in range(coast_dists.shape[1]):
    station_counts += feature_station.count_near(p_results.lats[:, i], p_results.lons[:, i])
    mouth_dists[:, i] = feature_mouth.get_all_dists(p_results.lats[:, i], p_results.lons[:, i])[0]
for i in range(coast_dists.shape[0]):
    for j in range(coast_dists.shape[1]):
        if np.isnan(p_results.traj[i, j]):
            coast_dists[i, j:] = np.nan
            break
        lat = p_results.lats[i, j]
        lon = p_results.lons[i, j]
        coast_dists[i, j] = feature_coast.get_closest_dist(lat, lon)

In [ ]:
d_mask = utils.conv_to_dataarray(utils.generate_mask_none(d_ds["u"].values), d_ds["u"].isel(time=0))

In [ ]:
def time_to_str(time):
    """
    haha time go chop chop
    
    Args:
        time (np.datetime64)
    """
    return str(time).split(".")[0]

### check whether a particle collided with land

whether it's within the defined distance with the coastline (in meters)

In [ ]:
p_results.plot_features = []
p_results.add_plot_feature(feature_station)

In [ ]:
p_results.plot_at_t(0, TIJUANA_MOUTH_DOMAIN)

In [ ]:
collided_land = np.zeros(p_ds["trajectory"].shape, dtype=bool)
lat_ls_coll = []
lon_ls_coll = []
times_sec = []
for p_iter in np.where((coast_dists <= collide_dist_thresh).sum(axis=1))[0]:
    p = p_ds["trajectory"][p_iter]
    # find the first state of the particle where it got stuck on land something
    for step in range(len(p)):
        if np.isnan(p[step]):
            break
        time = p_results.times[p_iter, step]
        time_secs = p_results.lifetimes[p_iter, step] + p_results.spawntimes[p_iter, step]
        lat = p_results.lats[p_iter, step]
        lon = p_results.lons[p_iter, step]
        times_sec.append(time_secs)
        # don't check if current data is nan anymore
        # check if it's close enough to the coastline
        near_coast = coast_dists[p_iter, step] <= collide_dist_thresh
        if near_coast:
            collided_land[p_iter, step] = True
            lat_ls_coll.append(lat)
            lon_ls_coll.append(lon)
            print(f"particle {p_iter} near coast at time {time_to_str(time)}")
            print(f"    spawn time: {time_to_str(p_results.times[p_iter, 0])}")
            print(f"    spawn coord: ({p_results.lats[p_iter, 0]}, {p_results.lons[p_iter, 0]})")
            print(f"    lifetime: {p_results.lifetimes[p_iter, step]}")
            # particle has hit coastline. we don't care about what happens after
            break

In [ ]:
# plot particles that have beached or something like that
domain = d_info.get_domain()
plot_particles(lat_ls_coll, lon_ls_coll, None, domain, land=True, part_size=30)
plt.plot(coastline_points.T[1], coastline_points.T[0])
plt.show()